In [46]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import time

def download_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, end=end_date)
    return data, stock.info

def calculate_standard_deviation(data):
    return data['Close'].pct_change().std() * np.sqrt(252)

def calculate_var(data, confidence_level=0.95):
    returns = data['Close'].pct_change().dropna()
    return np.percentile(returns, 100 * (1 - confidence_level))

def calculate_sharpe_ratio(data, risk_free_rate=0.02):
    returns = data['Close'].pct_change().dropna()
    excess_returns = returns - risk_free_rate / 252
    return np.sqrt(252) * excess_returns.mean() / excess_returns.std()

def calculate_sortino_ratio(data, risk_free_rate=0.02, target_return=0):
    returns = data['Close'].pct_change().dropna()
    downside_returns = returns[returns < target_return]
    excess_returns = returns - risk_free_rate / 252
    return np.sqrt(252) * excess_returns.mean() / downside_returns.std()

def calculate_max_drawdown(data):
    returns = data['Close'].pct_change().dropna()
    cumulative_returns = (1 + returns).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns / peak) - 1
    return drawdown.min()

def calculate_downside_deviation(data, target_return=0):
    returns = data['Close'].pct_change().dropna()
    downside_returns = returns[returns < target_return]
    return np.sqrt(np.mean(downside_returns**2))

def calculate_tracking_error(company_data, sector_index_data):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    return np.std(company_returns - index_returns) * np.sqrt(252)

def calculate_r_squared(company_data, sector_index_data):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    slope, intercept, r_value, _, _ = stats.linregress(index_returns, company_returns)
    return r_value**2

def calculate_treynor_ratio(company_data, beta, risk_free_rate=0.02):
    returns = company_data['Close'].pct_change().dropna()
    excess_returns = returns.mean() - risk_free_rate / 252
    return excess_returns / beta

def calculate_information_ratio(company_data, sector_index_data):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    return (company_returns.mean() - index_returns.mean()) / calculate_tracking_error(company_data, sector_index_data)

def calculate_cvar(data, confidence_level=0.95):
    returns = data['Close'].pct_change().dropna()
    var = calculate_var(data, confidence_level)
    return returns[returns <= var].mean()

def calculate_beta_adjusted_sharpe_ratio(company_data, sector_index_data, risk_free_rate=0.02):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    beta = stats.linregress(index_returns, company_returns).slope
    excess_returns = company_returns - risk_free_rate / 252
    return np.sqrt(252) * excess_returns.mean() / (excess_returns.std() * beta)

def calculate_drawdown_duration(data):
    returns = data['Close'].pct_change().dropna()
    cumulative_returns = (1 + returns).cumprod()
    drawdown = (cumulative_returns / cumulative_returns.cummax()) - 1
    drawdown_periods = (drawdown < 0).astype(int)
    return drawdown_periods.groupby((drawdown_periods != drawdown_periods.shift()).cumsum()).sum().max()

def calculate_ulcer_index(data):
    returns = data['Close'].pct_change().dropna()
    cumulative_returns = (1 + returns).cumprod()
    drawdown = (cumulative_returns / cumulative_returns.cummax()) - 1
    return np.sqrt(np.mean(drawdown**2))

def calculate_jensens_alpha(company_data, sector_index_data, risk_free_rate=0.02):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    beta = stats.linregress(index_returns, company_returns).slope
    expected_return = risk_free_rate + beta * (index_returns.mean() * 252 - risk_free_rate)
    return company_returns.mean() * 252 - expected_return


In [47]:

def main(ticker, sector_index_ticker="XLK", start_date="2012-01-01", end_date="2024-06-30"):
    company_data, company_info = download_stock_data(ticker, start_date, end_date)
    sector_index_data, _ = download_stock_data(sector_index_ticker, start_date, end_date)

    metrics = {
        'P/E Ratio': company_info.get('forwardEps') / company_info.get('previousClose') if company_info.get('forwardEps') and company_info.get('previousClose') else None,
        'P/B Ratio': company_info.get('priceToBook'),
        'Dividend Yield': company_info.get('dividendYield'),
        'Dividend Payout Ratio': company_info.get('payoutRatio'),
        'ROE': company_info.get('returnOnEquity'),
        'ROA': company_info.get('returnOnAssets'),
        'Beta': company_info.get('beta'),
        'Market Capitalization': company_info.get('marketCap'),
        'Revenue Growth': company_info.get('revenueGrowth'),
        'Debt-to-Equity Ratio': company_info.get('debtToEquity'),
        'Free Cash Flow': company_info.get('freeCashflow'),
        'Current Ratio': company_info.get('currentRatio'),
        'Quick Ratio': company_info.get('quickRatio'),
        'PEG Ratio': company_info.get('pegRatio'),
        'Standard Deviation': calculate_standard_deviation(company_data),
        'Value at Risk (VaR)': calculate_var(company_data),
        'Sharpe Ratio': calculate_sharpe_ratio(company_data),
        'Sortino Ratio': calculate_sortino_ratio(company_data),
        'Maximum Drawdown': calculate_max_drawdown(company_data),
        'Downside Deviation': calculate_downside_deviation(company_data),
        'Tracking Error': calculate_tracking_error(company_data, sector_index_data),
        'R-squared': calculate_r_squared(company_data, sector_index_data),
        'Treynor Ratio': calculate_treynor_ratio(company_data, beta=company_info.get('beta')),
        'Information Ratio': calculate_information_ratio(company_data, sector_index_data),
        'Conditional Value at Risk (CVaR)': calculate_cvar(company_data),
        'Beta-adjusted Sharpe Ratio': calculate_beta_adjusted_sharpe_ratio(company_data, sector_index_data),
        'Drawdown Duration': calculate_drawdown_duration(company_data),
        'Ulcer Index': calculate_ulcer_index(company_data),
        'Jensens Alpha': calculate_jensens_alpha(company_data, sector_index_data)
    }

    # Add metrics to the company data
    for metric, value in metrics.items():
        company_data[metric] = value

    # Save the data to a CSV file
    output_filename = f"{ticker}_data_with_metrics.csv"
    company_data.to_csv(output_filename)
    print(f"Data saved to {output_filename}")



In [48]:
if __name__ == "__main__":
    sp500=np.array(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ENPH', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'RE', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FMC', 'F', 'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN', 'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUM', 'HBAN', 'HII', 'IBM', 'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'INCY', 'IR', 'INTC', 'ICE', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JBHT', 'JKHY', 'J', 'JNJ', 'JCI', 'JPM', 'JNPR', 'K', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LDOS', 'LEN', 'LLY', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LUMN', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NFLX', 'NWL', 'NEM', 'NWSA', 'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NSC', 'NTRS', 'NOC', 'NLOK', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL', 'OMC', 'OKE', 'ORCL', 'OGN', 'OTIS', 'PCAR', 'PKG', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PBCT', 'PEP', 'PKI', 'PFE', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'QCOM', 'PWR', 'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SBUX', 'STT', 'STE', 'SYK', 'SIVB', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TGT', 'TEL', 'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TWTR', 'TYL', 'TSN', 'USB', 'UDR', 'ULTA', 'UAA', 'UA', 'UNP', 'UAL', 'UPS', 'URI', 'UNH', 'UHS', 'VLO', 'VTR', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VFC', 'VTRS', 'V', 'VNT', 'VNO', 'VMC', 'WRK', 'WAB', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WU', 'WRB', 'GWW', 'WY', 'WHR', 'WMB', 'WLTW', 'WYNN', 'XEL', 'XLNX', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS'])
    # sp500_tickers = s  # Replace with your array of tickers

    # Iterate through each ticker in the array
    for ticker in sp500:
        try:
            main(ticker)
            time.sleep(10)
        except Exception as e:
            print(f"An error occurred with ticker {ticker}: {e}")

Data saved to MMM_data_with_metrics.csv
Data saved to AOS_data_with_metrics.csv
Data saved to ABT_data_with_metrics.csv
An error occurred with ticker ABBV: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3141 and the array at index 1 has size 2891
Data saved to ACN_data_with_metrics.csv


$ATVI: possibly delisted; no timezone found
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=DMZZV4qzhQk


An error occurred with ticker ATVI: attempt to get argmax of an empty sequence
Data saved to ADM_data_with_metrics.csv
Data saved to ADBE_data_with_metrics.csv
Data saved to ADP_data_with_metrics.csv
Data saved to AAP_data_with_metrics.csv
Data saved to AES_data_with_metrics.csv
Data saved to AFL_data_with_metrics.csv
Data saved to A_data_with_metrics.csv
Data saved to APD_data_with_metrics.csv
Data saved to AKAM_data_with_metrics.csv
Data saved to ALK_data_with_metrics.csv
Data saved to ALB_data_with_metrics.csv
Data saved to ARE_data_with_metrics.csv
Data saved to ALGN_data_with_metrics.csv
An error occurred with ticker ALLE: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3141 and the array at index 1 has size 2669
Data saved to LNT_data_with_metrics.csv
Data saved to ALL_data_with_metrics.csv
Data saved to GOOGL_data_with_metrics.csv
Data saved to GOOG_data_with_metrics.csv
Data saved to MO_da

$ABC: possibly delisted; no timezone found
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=DMZZV4qzhQk


An error occurred with ticker ABC: attempt to get argmax of an empty sequence
Data saved to AME_data_with_metrics.csv
Data saved to AMGN_data_with_metrics.csv
Data saved to APH_data_with_metrics.csv
Data saved to ADI_data_with_metrics.csv
Data saved to ANSS_data_with_metrics.csv


...

An error occurred with ticker TWTR: attempt to get argmax of an empty sequence
Data saved to TYL_data_with_metrics.csv
Data saved to TSN_data_with_metrics.csv
Data saved to USB_data_with_metrics.csv
Data saved to UDR_data_with_metrics.csv
Data saved to ULTA_data_with_metrics.csv
Data saved to UAA_data_with_metrics.csv
An error occurred with ticker UA: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3141 and the array at index 1 has size 2080
Data saved to UNP_data_with_metrics.csv
Data saved to UAL_data_with_metrics.csv
Data saved to UPS_data_with_metrics.csv
Data saved to URI_data_with_metrics.csv
Data saved to UNH_data_with_metrics.csv
Data saved to UHS_data_with_metrics.csv
Data saved to VLO_data_with_metrics.csv
Data saved to VTR_data_with_metrics.csv
Data saved to VRSN_data_with_metrics.csv
Data saved to VRSK_data_with_metrics.csv
Data saved to VZ_data_with_metrics.csv
Data saved to VRTX_data

$WLTW: possibly delisted; no timezone found


An error occurred with ticker WLTW: attempt to get argmax of an empty sequence
Data saved to WYNN_data_with_metrics.csv
Data saved to XEL_data_with_metrics.csv


$XLNX: possibly delisted; no timezone found


An error occurred with ticker XLNX: attempt to get argmax of an empty sequence
Data saved to XYL_data_with_metrics.csv
Data saved to YUM_data_with_metrics.csv
Data saved to ZBRA_data_with_metrics.csv
Data saved to ZBH_data_with_metrics.csv
Data saved to ZION_data_with_metrics.csv
An error occurred with ticker ZTS: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3141 and the array at index 1 has size 2870
